In [1]:
from xml.dom.minidom import parse

In [2]:
dom = parse('camera_new.xml')
elem = dom.documentElement
elem

<DOM Element: document at 0x7f6b0c0ef3a0>

In [3]:
sensors = elem.getElementsByTagName('calibration')

In [4]:
# 默认字典值为空列表
from collections import defaultdict
intrinsics = defaultdict(list)

for sensor in sensors:
    resolution = sensor.getElementsByTagName('resolution')
    width = resolution[0].getAttribute('width')
    height = resolution[0].getAttribute('height')
    intrinsics['width'].append(float(width))
    intrinsics['height'].append(float(height))
    
    f = float(sensor.getElementsByTagName('f')[0].childNodes[0].nodeValue)
    intrinsics['f'].append(f)
    
    cx = sensor.getElementsByTagName('cx')[0].childNodes[0].nodeValue
    cx = float(width)/2 + float(cx)
    cy = sensor.getElementsByTagName('cy')[0].childNodes[0].nodeValue
    cy = float(height)/2 + float(cy)
    intrinsics['cx'].append(cx)
    intrinsics['cy'].append(cy)
    
    K = [f, 0, cx, 0, f, cy, 0, 0, 1]
    intrinsics['K'].append(K)
    
    # 径向畸变参数
    k1 = float(sensor.getElementsByTagName('k1')[0].childNodes[0].nodeValue)
    k2 = float(sensor.getElementsByTagName('k2')[0].childNodes[0].nodeValue)
    k3 = float(sensor.getElementsByTagName('k3')[0].childNodes[0].nodeValue)
#     radial_distortion = [k1, k2, k3]
#     intrinsics['radiDistort'].append(radial_distortion)
    
    # 切向畸变参数
    p1 = float(sensor.getElementsByTagName('p1')[0].childNodes[0].nodeValue)
    p2 = float(sensor.getElementsByTagName('p2')[0].childNodes[0].nodeValue)
#     circumferential_distortion = [p1, p2]
#     intrinsics['circumDistort'].append(circumferential_distortion)
    
    distort = [k1, k2, k3, p1, p2]
    intrinsics['distort'].append(distort)
    
intrinsics

defaultdict(list,
            {'width': [2300.0],
             'height': [2656.0],
             'f': [2263.536832402793],
             'cx': [1145.702476996091],
             'cy': [1329.734991823213],
             'K': [[2263.536832402793,
               0,
               1145.702476996091,
               0,
               2263.536832402793,
               1329.734991823213,
               0,
               0,
               1]],
             'distort': [[-0.08390679225628754,
               0.09327453495083804,
               -0.010616292149374984,
               0.0008191547611111152,
               0.0005511070668098991]]})

In [5]:
import re
def camera_sort(label):
    if re.search('21334181', label):
        return 0
    elif re.search('21334237', label):
        return 1
    elif re.search('21334180', label):
        return 2
    elif re.search('21334209', label):
        return 3
    elif re.search('21334208', label):
        return 4
    elif re.search('21334186', label):
        return 5
    elif re.search('21293326', label):
        return 6
    elif re.search('21293325', label):
        return 7
    elif re.search('21293324', label):
        return 8
    elif re.search('21334206', label):
        return 9
    elif re.search('21334220', label):
        return 10
    elif re.search('21334183', label):
        return 11
    elif re.search('21334207', label):
        return 12
    elif re.search('21334191', label):
        return 13
    elif re.search('21334184', label):
        return 14
    elif re.search('21334238', label):
        return 15
    elif re.search('21334221', label):
        return 16
    elif re.search('21334219', label):
        return 17
    elif re.search('21334190', label):
        return 18
    elif re.search('21334211', label):
        return 19

In [6]:
import numpy as np
from icecream import ic

frame_num = '49_50_'
cameras = elem.getElementsByTagName('camera')

out = dict()
for i in range(20):
    out['cam'+str(i)] = dict()

for camera in cameras:
    parameters = dict()
    
    # 属于哪一组的摄像机
    sensor_id = camera.getAttribute('sensor_id')
    sensor_id = int(sensor_id)
    sensor_id = 0

    print(sensor_id)
    parameters['K'] = intrinsics['K'][sensor_id]
    
    label = camera.getAttribute('label')
    cam_num = camera_sort(label)
    
    transform = camera.getElementsByTagName('transform')[0].childNodes[0].nodeValue
    transform = transform.split(' ')
    transform = [float(i) for i in transform]
    
    transform = np.array(transform)
    transform = transform.reshape(4,4)
    # 外参数矩阵为transfrom的逆矩阵
    # print(transform)
    extrinsics = np.linalg.inv(transform)
    # extrinsics = transform

    # ic(transform@extrinsics)

    R = extrinsics[0:3,0:3].reshape(-1).tolist()
    T = extrinsics[0:3,-1].tolist()
    
    parameters['R'] = R
    parameters['T'] = T
    parameters['imgSize'] = [intrinsics['width'][sensor_id], intrinsics['height'][sensor_id]]
    parameters['distort'] = intrinsics['distort'][sensor_id]
    
    out['cam'+str(cam_num)] = parameters

print(out)
# cameras.getElementsByTagName('camera')[0].getAttribute('label')

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
{'cam0': {'K': [2263.536832402793, 0, 1145.702476996091, 0, 2263.536832402793, 1329.734991823213, 0, 0, 1], 'R': [-0.6681006071358447, -0.32203385738774953, 0.6707725198908207, -0.48093977428810425, -0.5009658091804041, -0.7195347048891598, 0.5677486347428959, -0.8033227575067131, 0.17981667058253653], 'T': [0.92216149982717, -0.9235894596007, 1.5334185870631467], 'imgSize': [2300.0, 2656.0], 'distort': [-0.08390679225628754, 0.09327453495083804, -0.010616292149374984, 0.0008191547611111152, 0.0005511070668098991]}, 'cam1': {'K': [2263.536832402793, 0, 1145.702476996091, 0, 2263.536832402793, 1329.734991823213, 0, 0, 1], 'R': [-0.6616755894407668, -0.32070349314184804, 0.6777423432432351, -0.25165490452348566, -0.7564865103944719, -0.6036538483439591, 0.7062968379990738, -0.5699802006015756, 0.41984205072229047], 'T': [0.9273381890786974, -0.8327513223559815, 1.6860849578711445], 'imgSize': [2300.0, 2656.0], 'distort': [-0.08390679225628754, 0.09327453

In [7]:
import json
with open("camera_new.json", "w", encoding='utf-8') as f:
    json.dump(out, f, indent=4)